<a href="https://colab.research.google.com/github/spinto88/Clases_y_tutoriales/blob/main/Web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Trabajando con el contenido estático de una página

Vamos a comenzar trabajando haciendo un simple request a una página, obteniendo así el código HTML de la misma.

Al hacer un request estamos sacando un foto de la página: su contenido es completamente estático. Por estático nos referimos a que no vamos a tener habilitadas opciones como *scrollear* o bien clickear algún que otro botón. Esto último es importante porque muchas veces hay contenido que solo se puede acceder a través de dichos métodos. Trabajar con contenido dinámico lo vamos a ver aparte de esta notebook (en caso de necesitar, cuando trabajemos, por ejemplo, con la librería *selenium*).

Dado que sacamos una foto de la página, recibimos un código que vamos a tener que *parsearlo*, es decir, convertirlo en un objeto manipulable en el que sea fácil navegar. Esto lo hacemos con el módulo *BeautifulSoup*.



In [ ]:
# Importamos las librerías que vamos a utilizar
import requests
from bs4 import BeautifulSoup as BS
import time

Comencemos como prueba con el *homepage* del diario La Nación. Recordar que una vez que hacemos el request ya dejamos de interactuar con la página:

In [ ]:
# Hacemos un request a la página de La Nación
response = requests.get("https://www.lanacion.com.ar")

# Vemos el contenido que nos devolvió
print(response.status_code)
print(response.content)

200
b'<!DOCTYPE html><html lang="es"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=0.5,maximum-scale=5.0,user-scalable=yes"/><meta name="theme-color" content="#ffffff"/><title>\xc3\x9altimas noticias de Argentina y el mundo - LA NACION</title><link rel="preload" as="image" fetchPriority="high" media="(min-width: 768px)" href="https://www.lanacion.com.ar/resizer/v2/juan-schiaretti-y-sergio-massa-en-el-segundo-Y6OVNITB3FF3XFITF3T73AGHFU.jpg?auth=8b161b934a382db1fd99319b003ea190708953cd64c6f6cd8a9e6fb8d132bf22&amp;width=488&amp;height=325&amp;quality=70&amp;smart=true"/><link rel="preload" as="image" fetchPriority="high" media="(max-width: 767px)" href="https://www.lanacion.com.ar/resizer/v2/juan-schiaretti-y-sergio-massa-en-el-segundo-Y6OVNITB3FF3XFITF3T73AGHFU.jpg?auth=8b161b934a382db1fd99319b003ea190708953cd64c6f6cd8a9e6fb8d132bf22&amp;width=420&amp;height=280&amp;quality=70&amp;smart=true"/><style>\n@font-face {font-fami

**Observación**: el *status_code* es un número que nos dice cómo nos fue al querer conectarnos con la página. *200* significa todo está bien!. Probablemente estan familiarizados con el *404* que aparece cuando una página no existe. Más info aquí: https://www.w3schools.com/tags/ref_httpmessages.asp

Listo! La información que nos dió el request es todo con lo que contamos. Ahora no queda otra que arremangarse y trabajar con esto. Para eso, *BeautifulSoup* nos da bastantes facilidades. Primero creamos la sopa:

In [ ]:
# soup es un objeto manipulable creado a partir del contenido de la página y BeautifulSoup
soup = BS(response.content)

print(soup)

<!DOCTYPE html>
<html lang="es"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1.0,minimum-scale=0.5,maximum-scale=5.0,user-scalable=yes" name="viewport"/><meta content="#ffffff" name="theme-color"/><title>Últimas noticias de Argentina y el mundo - LA NACION</title><link as="image" fetchpriority="high" href="https://www.lanacion.com.ar/resizer/v2/juan-schiaretti-y-sergio-massa-en-el-segundo-Y6OVNITB3FF3XFITF3T73AGHFU.jpg?auth=8b161b934a382db1fd99319b003ea190708953cd64c6f6cd8a9e6fb8d132bf22&amp;width=488&amp;height=325&amp;quality=70&amp;smart=true" media="(min-width: 768px)" rel="preload"/><link as="image" fetchpriority="high" href="https://www.lanacion.com.ar/resizer/v2/juan-schiaretti-y-sergio-massa-en-el-segundo-Y6OVNITB3FF3XFITF3T73AGHFU.jpg?auth=8b161b934a382db1fd99319b003ea190708953cd64c6f6cd8a9e6fb8d132bf22&amp;width=420&amp;height=280&amp;quality=70&amp;smart=true" media="(max-width: 767px)" rel="preload"/><style>
@font-face {font-family:'Prumo';sr

Ahora todo lo que queda es encontrar los elementos que nos interese.
¿Cómo los identificamos? Es un trabajo de ida y vuelta, y prueba y error, así que a no frustrarse si no sale de entrada. Los pasos a seguir son:
- Ir a la página que queremos scrapear y con el botón derecho del mouse poner "inspeccionar elemento" sobre el sector de la página que nos interese.
- Luego podemos identificarlo por su *tag* o por algún atributo que nos permita identificarlo mejor, como por ejemplo, la clase (*class*).
- A veces puede ser más conveniente no apuntarle al elemento que nos interesa sino a un nodo superior (que lo contenga) y a partir de ahí navegar hacia dentro.

Investigando la página del diario La Nación encontramos que los recuadros correspondientes a los títulos y enlaces de las principales notas están dentro de bloques con atributo. Por ejemplo, algo del estilo:
~~~
<... class = "com-title --xl">
~~~
o
~~~
<... class = "com-title --xs">
~~~
(leer esta notebook a la par que de la inspección de la página. Las páginas se van actualizando y por lo tanto la forma del código HTML puede variar).

Como observamos que el tag puede ser *h1* o *h2*, vamos a tratar de no especificar el *tag* para no perdernos nada. Buscamos entonces todos los elementos que cumplan con alguno de estos requisitos sin especificar el *tag*:

In [ ]:
# Creamos una lista con los elementos cuya clase sea igual a lo que encontremos en la página, con cualquier tag
elements = soup.findAll(attrs = {"class": "description-container --relative flex flex-column"})

Vemos en qué consiste la lista de elementos:

In [ ]:
len(elements)

125

In [ ]:
elements[2]

<section class="description-container --relative flex flex-column" href="/politica/presupuesto-2024-ganadores-y-perdedores-del-reparto-de-planes-sociales-que-proyecto-sergio-massa-nid03112023/"><h2 class="text ln-text title --prumo --font-medium --font-m-l"><span class="text ln-text lead --prumo --font-extra">Sin criterios objetivos.<!-- --> </span>Ganadores y perdedores del reparto de planes sociales que proyectó Massa para 2024</h2><div class="marquee-container flex ai-center" title="Por Gabriela Origlia"><div class="marquee-text flex flex-column --font-2xs w-100"><span class="text ln-text marquee">Por <strong>Gabriela Origlia</strong></span></div></div></section>

Veamos qué tenemos en algún elemento de ejemplo:

In [ ]:
element = elements[0]

element

<section class="description-container --relative flex flex-column" href="/politica/fuerte-mensaje-de-juan-schiaretti-contra-sergio-massa-por-el-juicio-politico-a-la-corte-suprema-nid03112023/"><h1 class="text ln-text title --prumo --font-medium --font-4xl"><span class="text ln-text lead --prumo --font-extra">En defensa de la Corte.<!-- --> </span>Fuerte mensaje de Schiaretti: "El gobierno kirchnerista del ministro Sergio Massa"</h1><h2 class="text ln-text subhead --font-s">El gobernador cuestionó al oficialismo y pidió respetar la división de poderes; a poco más de dos semanas del balotaje, el candidato de Unión por la Patria se disputa con Milei los votos del mandatario cordobés </h2></section>

Vemos por ejemplo como sacar el texto aquí adentro y el enlace a la nota:

In [ ]:
# Todo el texto contenido dentro del elemento
print(element.text)

En defensa de la Corte. Fuerte mensaje de Schiaretti: "El gobierno kirchnerista del ministro Sergio Massa"El gobernador cuestionó al oficialismo y pidió respetar la división de poderes; a poco más de dos semanas del balotaje, el candidato de Unión por la Patria se disputa con Milei los votos del mandatario cordobés 


In [ ]:
# Enlace a la nota (o parte del enlace, a veces es relativo a la página)
# Buscamos un tag "a" y el atributo "href"
print(element.get('href'))

/politica/fuerte-mensaje-de-juan-schiaretti-contra-sergio-massa-por-el-juicio-politico-a-la-corte-suprema-nid03112023/


Como cada elemento puede tener un enlace a la página (*href*) contenido dentro de un *tag < a >*, lo que podemos hacer es iterar sobre todos los elementos y encontrar el *tag a* y pedirle el valor del atributo *href*:

In [ ]:
# Iteramos sobre los elementos
# Hacemos un try ... except ... por si alguno de los elementos no tiene enlace
# (puede ser que estemos agarrando más cosas de las que nos interesa)
urls = []
for element in elements:
  try:
    urls.append(element.get('href'))
  except:
    pass

print(urls)

['/politica/fuerte-mensaje-de-juan-schiaretti-contra-sergio-massa-por-el-juicio-politico-a-la-corte-suprema-nid03112023/', '/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-de-gaza-y-nid03112023/', '/politica/presupuesto-2024-ganadores-y-perdedores-del-reparto-de-planes-sociales-que-proyecto-sergio-massa-nid03112023/', '/tecnologia/un-yacimiento-de-hidrogeno-blanco-podria-disparar-una-revolucion-energetica-nid02112023/', '/economia/la-caida-de-ingresos-llegaria-al-14-este-ano-el-mayor-descenso-en-20-anos-nid03112023/', '/deportes/salio-a-la-luz-el-gesto-que-lionel-messi-le-hizo-a-antonela-roccuzzo-desde-el-escenario-al-recibir-nid03112023/', '/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-de-gaza-y-nid03112023/', '/economia/dolar/dolar-hoy-dolar-blue-hoy-a-cuanto-cotiza-este-viernes-3-de-noviembre-nid03112023/', '/el-mundo/tras-su-operacion-de-cadera-lula-se-mostro-ejercitando-en-b

In [ ]:
print(len(urls))

125


Ya tenemos enlaces de páginas (¿son todos los que vemos cuando abrimos la página? ¿No habrá contenido dinámico que nos estamos perdiendo?).

### Scrappeo de la página de un artículo

Veamos si podemos extraer contenido de las notas cuyos enlaces extraímos. La idea es la misma: ir a la página e inspeccionar los elementos que nos interese. Cuando las páginas corresponden a la misma plataforma es usual que la información se encuentre en el mismo lugar, por lo que aprendamos de una sola paǵina podemos quizás extrapolarlo a otras.

Agarremos un *url* (por ejemplo, alguno de los de arriba) y generemos la sopa:

In [ ]:
# Url de interés
url = 'https://www.lanacion.com.ar/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-de-gaza-y-nid03112023/'

# Hacemos un request a dicha página
response = requests.get(url)
print(response.status_code)

# Creamos la sopa para manipular
soup = BS(response.content)

200


Identificando dónde está el título de la nota (eso de vuelta, inspeccionando el elemento asociado), podemos extraer el texto.
Es posible que haya muchas formas de encontrar un mismo elemento, basta con ser lo más preciso posible para obtener la información que querramos, pero lo suficientemente general para que sea extrapolable a todas las páginas.

In [ ]:
# Buscamos el elemento cuya clase indentificamos como correspondiente al título de la nota
title = soup.find(attrs = {"class": "com-title --font-primary --sixxl --font-extra"}) # Funcionó sólo con el nombre del tag, pero podríamos haber especificado los atributos de clase(attrs = {"class": "com-title --threexl"})
print(title.text)

El líder de Hezbollah dice que el ataque de Hamas fue “100% palestino”, pero advierte sobre una posible guerra regional: “Todas las opciones están sobre la mesa”


Fecha de la nota (de nuevo, inspeccionar la página hasta dar en la tecla de dónde está esta información):

In [ ]:
date = soup.find(attrs = {"class": "com-date --twoxs"})
print(date.text)

3 de noviembre de 2023


O, urgando un poco más en el código HTML, podemos extraer la fecha mejor formateada (notar otra manera de buscar cosas con BS):

In [ ]:
# Buscamos un tag del tipo "meta", cuyo atributo "property" tiene valor "article:published_time"
# A este tag le sacamos el valor que toma el atributo "content"
datetime = soup.find("meta", attrs = {"property": "article:published_time"})['content']
print(datetime)

2023-11-03T15:15:43.952Z


Veamos la descripción de la nota, que podemos encontrarla en:

In [ ]:
description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs-"})
print(description.text)

Hassan Nasrallah brindó un extenso discurso por primera vez desde que estalló el conflicto y apuntó contra Estados Unidos mientras Antony Blinken está en Tel Aviv para pedir una pausa en la ofensiva israelí


Por último veamos el cuerpo de la nota. Encontramos un nodo padre del tipo *section* cuya clase es *cuerpo__nota* (notar que tenemos que escribir todo exactamente como lo vemos en el código de la página. Cualquier error de tipeo va a resultar en que no nos devuelva nada):

In [ ]:
# Buscamos un tag del tipo "section", cuyo atributo "class" tiene valor "cuerpo__nota"
body = soup.find("section", attrs = {"class": "cuerpo__nota"})

Usualmente, los párrafos están dentro de tags denominados *p*. Por lo que buscamos todos estos dentro del bloque asociado al cuerpo de la nota:

In [ ]:
# Párrafos usualmente dentro de tags p
paragraphs = body.findAll('p')

print(paragraphs)

[<p class="com-paragraph --capital --s">KHAN YOUNIS, Franja de Gaza.- El líder de Hezbollah,<strong> Hassan Nasrallah</strong>, sorprendió hoy con un enérgico discurso en el que<strong> subrayó que “todas las opciones están sobre la mesa” </strong>en el marco de <a class="com-link" data-reactroot="" href="https://www.lanacion.com.ar/el-mundo/guerra-en-medio-oriente-israel-combate-en-dos-frentes-y-empieza-a-sentir-la-feroz-resistencia-de-nid02112023/" target="_self" title="la guerra en Gaza">la guerra en Gaza</a>, avivando el temor a que el conflicto se extienda a la región, al tiempo que <strong>acusó a Estados Unidos de ser responsable de los “crímenes perpetrados por Israel</strong>”.</p>, <p class="com-paragraph --s">En una aparición televisada de casi una hora y media, Nasrallah elogió en repetidas ocasiones la sanguinaria ofensiva perpetrada por Hamas el 7 de octubre, una operación que, según sus palabras, fue<strong> “decidida e implementada al 100% por palestinos”. </strong>Fue 

In [ ]:
paragraphs[0].text

'KHAN YOUNIS, Franja de Gaza.- El líder de Hezbollah, Hassan Nasrallah, sorprendió hoy con un enérgico discurso en el que subrayó que “todas las opciones están sobre la mesa” en el marco de la guerra en Gaza, avivando el temor a que el conflicto se extienda a la región, al tiempo que acusó a Estados Unidos de ser responsable de los “crímenes perpetrados por Israel”.'

In [ ]:
# Iteramos para todo párrafo y vemos el texto contenido
# Podemos hacer una lista y concatenar todo con un salto de línea
texts = []
for p in paragraphs:
  texts.append(p.text)

body_text = '\n'.join(texts)

print(body_text)

KHAN YOUNIS, Franja de Gaza.- El líder de Hezbollah, Hassan Nasrallah, sorprendió hoy con un enérgico discurso en el que subrayó que “todas las opciones están sobre la mesa” en el marco de la guerra en Gaza, avivando el temor a que el conflicto se extienda a la región, al tiempo que acusó a Estados Unidos de ser responsable de los “crímenes perpetrados por Israel”.
En una aparición televisada de casi una hora y media, Nasrallah elogió en repetidas ocasiones la sanguinaria ofensiva perpetrada por Hamas el 7 de octubre, una operación que, según sus palabras, fue “decidida e implementada al 100% por palestinos”. Fue su primer mensaje público desde que comenzó la guerra.
🔴 “LIVE: Sayyed Hassan Nasrallah, the Secretary General of Lebanon’s Hezbollah resistance movement, delivers speech https://t.co/jr84oCDLZz
En este sentido, Nasrallah afirmó que lo sucedido confirma que “Irán no ejerce ninguna tutela sobre las facciones de resistencia”.
Nasrallah sostuvo durante su discurso que la ofensiva

Si vemos que falta parte del texto, tenemos que identificar dónde le erramos: puede ser que el texto no esté dentro del párrafo, por lo que tendríamos que ver qué tag tiene asociado, con qué clase o atributo que nos permita acceder.

### Iteración sobre varios urls

Una de las cosas interesantes del scrapeo es poder iterar sobre varias páginas y extraer información en forma sistemática. Extraigamos el título y la descripción para un conjunto de urls. Básicamente es hacer lo que hicimos para una nota varias veces:

In [ ]:
# Recordamos teníamos un listado de urls
urls

['/politica/fuerte-mensaje-de-juan-schiaretti-contra-sergio-massa-por-el-juicio-politico-a-la-corte-suprema-nid03112023/',
 '/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-de-gaza-y-nid03112023/',
 '/politica/presupuesto-2024-ganadores-y-perdedores-del-reparto-de-planes-sociales-que-proyecto-sergio-massa-nid03112023/',
 '/tecnologia/un-yacimiento-de-hidrogeno-blanco-podria-disparar-una-revolucion-energetica-nid02112023/',
 '/economia/la-caida-de-ingresos-llegaria-al-14-este-ano-el-mayor-descenso-en-20-anos-nid03112023/',
 '/deportes/salio-a-la-luz-el-gesto-que-lionel-messi-le-hizo-a-antonela-roccuzzo-desde-el-escenario-al-recibir-nid03112023/',
 '/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-de-gaza-y-nid03112023/',
 '/economia/dolar/dolar-hoy-dolar-blue-hoy-a-cuanto-cotiza-este-viernes-3-de-noviembre-nid03112023/',
 '/el-mundo/tras-su-operacion-de-cadera-lula-se-mostro-ejercita

Ojo, en este caso, los url que encontramos NO son aquellos que nos permiten directamente acceder al contenido de las páginas, sino que tenemos que anteponerles el dominio de la página (esta práctica es frecuente en muchas páginas)

In [ ]:
# Lo siguiente se llama lista por compresión, es una forma rápida de definir una lista iterando sobre cosas
urls_complete = ['https://www.lanacion.com.ar' + url for url in urls]
urls_complete

['https://www.lanacion.com.ar/politica/fuerte-mensaje-de-juan-schiaretti-contra-sergio-massa-por-el-juicio-politico-a-la-corte-suprema-nid03112023/',
 'https://www.lanacion.com.ar/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-de-gaza-y-nid03112023/',
 'https://www.lanacion.com.ar/politica/presupuesto-2024-ganadores-y-perdedores-del-reparto-de-planes-sociales-que-proyecto-sergio-massa-nid03112023/',
 'https://www.lanacion.com.ar/tecnologia/un-yacimiento-de-hidrogeno-blanco-podria-disparar-una-revolucion-energetica-nid02112023/',
 'https://www.lanacion.com.ar/economia/la-caida-de-ingresos-llegaria-al-14-este-ano-el-mayor-descenso-en-20-anos-nid03112023/',
 'https://www.lanacion.com.ar/deportes/salio-a-la-luz-el-gesto-que-lionel-messi-le-hizo-a-antonela-roccuzzo-desde-el-escenario-al-recibir-nid03112023/',
 'https://www.lanacion.com.ar/el-mundo/israel-entre-la-tension-por-un-posible-nuevo-frente-con-hezbollah-el-cerco-sobre-la-ciudad-d

In [ ]:
# Barremos en el listado de urls
# Le ponemos un try-except para ignorar errores
for url in urls_complete:

  try:

    # Hacemos request al url actual
    response = requests.get(url)

    # Creamos la sopa
    soup = BS(response.content)

    # Identificamos el título
    title = soup.find(attrs = {"class": "com-title --font-primary --sixxl --font-extra"}).text

    # Identificamos la descripción
    description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs-"}).text

    # Printeamos ambas y dejamos un espacio entre nota y nota
    print(title)
    print(description)
    print('\n\n')

  except:
    pass

Fuerte mensaje de Juan Schiaretti contra Sergio Massa por el juicio político a la Corte Suprema
El gobernador cuestionó al oficialismo y pidió respetar la división de poderes; a poco más de dos semanas del balotaje, el candidato de Unión por la Patria se disputa con Milei los votos del mandatario cordobés 



El líder de Hezbollah dice que el ataque de Hamas fue “100% palestino”, pero advierte sobre una posible guerra regional: “Todas las opciones están sobre la mesa”
Hassan Nasrallah brindó un extenso discurso por primera vez desde que estalló el conflicto y apuntó contra Estados Unidos mientras Antony Blinken está en Tel Aviv para pedir una pausa en la ofensiva israelí



Presupuesto 2024 | Ganadores y perdedores del reparto de planes sociales que proyectó Sergio Massa para 2024
Jujuy, Chaco, Tucumán y la provincia de Buenos Aires son las provincias más favorecidas con el plan Potenciar Trabajo, según lo proyectado por el ministro-candidato 



Un yacimiento de hidrógeno blanco “casi

Genial, obtuvimos información de las principales noticias que están en la página principal de La Nación (¿se animan a agregarle la sección, el autor o autora de la nota si lo hubiere, etc.?)



### Guardado de la información

Si bien scrapear puede ser medio engorroso, lo bueno es que quizás solo hace falta hacerlo una única vez, es decir, una vez que obtenemos los datos que nos interesan, los guardamos y hacemos todos los análisis que nos interese.

In [ ]:
# Creamos listas vacías para ir guardando la información
titles = []
descriptions = []
dates = []
bodies = []

# Barremos en el listado de urls
# Le ponemos un try-except para ignorar errores
for url in urls_complete:

  try:

    # Hacemos request al url actual
    response = requests.get(url)

    # Creamos la sopa
    soup = BS(response.content)

    # Identificamos el título
    title = soup.find(attrs = {"class": "com-title --font-primary --sixxl --font-extra"}).text

    # Identificamos la descripción
    description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs-"}).text

    # Cuerpo de la nota
    body = soup.find("section", attrs = {"class": "cuerpo__nota"})
    paragraphs = body.findAll('p')
    # Juntamos el texto de los párrafos, separados por un salto de línea
    paragraphs_text = '\n'.join([p.text for p in paragraphs])

    # Fecha
    datetime = soup.find("meta", attrs = {"property": "article:published_time"})['content']

    # En vez de printear guardamos los datos en las listas que creamos
    titles.append(title)
    descriptions.append(description)
    dates.append(datetime)
    bodies.append(paragraphs_text)

  except:
    pass

Con las listas con toda la información, creamos un dataframe de pandas que para meter todo ahí y guardamos el archivo en un .csv para que quede disponible para futuros análisis.

In [ ]:
import pandas as pd

# Creamos un dataframe vacío
df = pd.DataFrame()

df['title'] = titles
df['description'] = descriptions
df['date'] = dates
df['body'] = bodies

# Vemos las primeras líneas
df.head()

,title,description,date,body
0,La insólita explicación de Javier Milei por su...,El economista planteó que su bloque solo parti...,2022-10-26T17:14:25.489Z,Luego de las críticas de los dirigentes de Jun...
1,“Francamente inaceptable”: Cristina Kirchner c...,La vicepresidenta utilizó su cuenta de Twitter...,2022-10-26T14:16:00.622Z,Justo después de que terminara la entrevista q...
2,Una diputada que sufrió la muerte de su bebé y...,La legisladora Camila Crescimbeni se largó a l...,2022-10-26T16:53:38.098Z,Ya terminaba el debate por el Presupuesto 2023...
3,Cuentas de los argentinos en EE. UU.: asado de...,Massa estuvo ayer con el senador Bob Menendez ...,2022-10-26T15:59:00.204Z,"Sergio Massa se acostó esta mañana, luego de l..."
4,Qué necesita Buenos Aires para convertirse en ...,"El urbanista Carlos Moreno, impulsor del conce...",2022-10-26T14:20:02.724Z,Hay un concepto que los urbanistas del gobiern...


In [ ]:
# Guardamos el archivo que podemos descargar
df.to_csv('InfoLaNacion.csv', index = False)

### Tarea para el hogar

- Encuentre una paǵina de interés y trate de extraer información que le interese (no todas son fáciles de hacer, no se frustre si falla...). Ejemplo, pueden ser páginas con letras de canciones, recetas de cocina, artículos en blogs, etc.
- Si tiene un conjunto de páginas, guarde la información relevante en dataframe de pandas para poder analizarla. Por ejemplo, puede hacer descomposición en tópicos o análisis de sentimiento de textos descargados.